In [1]:
%cd spark/hw-3

/home/vdubyna/Workspace/Jupyter/spark/hw-3


In [2]:
%ls

data/  spark.ipynb


In [6]:
datafile = '/home/vdubyna/Workspace/Jupyter/spark/hw-3/data/nuek-vuh3.csv'

In [5]:
import random
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder
    .appName("MyGoitSparkSandbox")
    .master("spark://10.0.1.28:7077")
    .getOrCreate()
)
nuek_df = spark.read.csv(datafile, header=True)

# nuek_alt_read_df = (spark.read.format("csv")
#                     .option("header", "true")
#                     .load(datafile))

# Виводимо перші 10 записів
# nuek_df.show(10)

# Створюємо тимчасове представлення для виконання SQL-запитів
nuek_df.createTempView("nuek_view")
# Виконуємо SQL-маніпуляції
spark.sql("SELECT * FROM nuek_view LIMIT 10").show()

# Закриваємо сесію Spark
spark.stop()

+-----------+-------+---------------+----------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------+-------------+----------------------+--------------------+--------------------+----+-------------------+---------+------------+----+-----------------+--------+--------------+--------+---------------+----------------+---------+------------------------------+------------------------+-------------------+---------------------------------+-------------+--------------------+--------------------+--------------------+---------------------------+
|call_number|unit_id|incident_number|       call_type|           call_date|          watch_date|       received_dttm|          entry_dttm|       dispatch_dttm|       response_dttm|       on_scene_dttm|transport_dttm|hospital_dttm|call_final_disposition|      available_dttm|             address|city|zipcode_of_incident|battalion|station_area

In [7]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, min, max, avg, unix_timestamp, count_if, round, when
from pyspark.sql.types import TimestampType, IntegerType

In [12]:
spark = (SparkSession.builder.appName("MyGoitSparkSandbox").master("spark://10.0.1.28:7077").getOrCreate())

# Завантажуємо датасет
nuek_df = spark.read.csv(datafile, header=True)

# Створюємо тимчасове представлення для виконання SQL-запитів
nuek_df.createTempView("nuek_view")

# Скільки унікальних call_type є в датасеті?
print(nuek_df.select('call_type')
      .where(col("call_type").isNotNull())
      .distinct()
      .count())

nuek_df.select('call_type') \
    .where(col("call_type").isNotNull()) \
    .distinct() \
    .explain()
spark.sql("""
    SELECT COUNT(DISTINCT call_type) as count
    FROM nuek_view
    WHERE call_type IS NOT NULL
""").explain()

# Скільки унікальних call_type є в датасеті? (з використанням SQL)
df = spark.sql("""SELECT COUNT(DISTINCT call_type) as count
                    FROM nuek_view
                    WHERE call_type IS NOT NULL""")
# Виводимо датафрейм на дисплей
df.show()

# Витягуємо дані колонки з датафрейму
print(df.collect(), type(df.collect()))
# Дотягуємось до самого значення за номером рядка та іменем колонки
print(df.collect()[0]['count'])
# або за номером рядка та номером колонки
print(df.collect()[0][0])

df = spark.sql("""SELECT DISTINCT call_type
FROM nuek_view
WHERE call_type IS NOT NULL""")
# Виводимо датафрейм на дисплей
df.show()

spark.stop()

16
== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[call_type#1800], functions=[])
   +- Exchange hashpartitioning(call_type#1800, 200), ENSURE_REQUIREMENTS, [plan_id=1235]
      +- HashAggregate(keys=[call_type#1800], functions=[])
         +- Filter isnotnull(call_type#1800)
            +- FileScan csv [call_type#1800] Batched: false, DataFilters: [isnotnull(call_type#1800)], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/home/vdubyna/Workspace/Jupyter/spark/hw-3/data/nuek-vuh3.csv], PartitionFilters: [], PushedFilters: [IsNotNull(call_type)], ReadSchema: struct<call_type:string>


== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[], functions=[count(distinct call_type#1800)])
   +- Exchange SinglePartition, ENSURE_REQUIREMENTS, [plan_id=1266]
      +- HashAggregate(keys=[], functions=[partial_count(distinct call_type#1800)])
         +- HashAggregate(keys=[call_type#1800], functions=[])
            +- Exchange hashp